In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.stats.api as sms

# Načítanie dát

In [4]:
X_train = pd.read_csv('clean_data/X_train.csv', on_bad_lines='warn', delimiter=',')
X_test = pd.read_csv('clean_data/X_test.csv', on_bad_lines='warn', delimiter=',')
y_train = pd.read_csv('clean_data/y_train.csv', on_bad_lines='warn', delimiter=',')
y_test = pd.read_csv('clean_data/y_test.csv', on_bad_lines='warn', delimiter=',')

,Unnamed: 0,mwra
6,20427,0.0
8,46269,0.0
10,41604,0.0
11,30124,0.0
14,19951,0.0
...,...,...
9658,11444,0.0
9660,7416,0.0
9666,12980,0.0
9667,22155,0.0
